In [ ]:
import numpy as np
from scripts import *
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable
path = './data/thermodyn/'
if os.path.exists('./data_figshare/thermodyn/'):
    path='./data_figshare/thermodyn/'

In [ ]:
# Calibration of the crustal density and degree of depletion of the continentlal lithospheric mantle
# and sensitivity on water depth/sea level
#

water_depth_at_ridge     = 2750.0e0  # meters = reference water depth at the ridge based on average MOR elevation
dz                       = 200e0     # meters
ref_depth                = 125e3     # meters = compensation depth (base of the continental lithosphere)

# Lithosphere 1 = Reference MOR column (geotherm and densities from model M1/M2)
# The MOR from 2-D thermechanical model is used to calibrate density structure of the reference continental column 
#
reflith                 = 'MOR'
book_reflith            = book_ridge      # parameters (./scripts/book_lith.py)

# Lithosphere 2 = Reference continental column
complith                = 'lith125'
book_complith           = book_lith125_GL # parameters (./scripts/book_lith.py)
perplex_name            = 'slm_sp2008'    # composition of the continental lithospheric mantle


use_fixed_waterdepth    = False           # True means that sea level can change but that waterdepth at the ridge is fixed


######## COMMENT/UNCOMMENT OR DEFINE A CASE #########################################################

# Case1: Reference Case Perplex
# => Calibration of the crustal density and degree of depletion of the continentlal lithospheric mantle
# slow... (outputs are provided - next cell)
printOut                 = False          # print more information
use_perplex_table_mantle = True
allrefdensity_crust      = np.arange(2750,2960,10)
alllith_depletion        = np.arange(-30,2,2)
allsea_level_ref         = [-400]

# Case 2: Temperature dependent density
# => Check sensitivity on elevation in the reference case (Reference density in the mantle 3311 kg/m3)
# slow... (outputs are provided - next cell)
printOut                 = False          # print more information
use_perplex_table_mantle = False
allrefdensity_crust      = np.arange(2750,2960,10)
alllith_depletion        = np.arange(-30,2,2)
allsea_level_ref         = [-400]

# Case 3: Single Case
#
printOut                 = True
use_perplex_table_mantle = False
allrefdensity_crust      = [2860]
alllith_depletion        = [-13]
allsea_level_ref         = [-200]

############################################################################################


if use_perplex_table_mantle:
    # (see ./scripts/scripts_geodyn1d.py)
    # - turn on reading perplex grid option
    # - define the correct perplex grid
    # - use the right degree of depletion in the continental lithospheric mantle to fit MOR/continets elevation
    #
    book_reflith  = update_mantle_density_to_reference(reflith, book_reflith, perplex_name=perplex_name)  
    book_complith = update_mantle_density_to_reference(complith,book_complith,perplex_name=perplex_name)
else:
    # The reference density in the mantle is 3311 kg/m3 (see calibration below)
    book_reflith  = update_mantle_density_to_reference(reflith, book_reflith, perplex_name=None)  
    book_complith = update_mantle_density_to_reference(complith,book_complith,perplex_name=None)

if printOut:
    print("\n# Column 1: Reference MOR ###################################################################################")
if use_perplex_table_mantle:
    deltarho_ridge_correction        = -0.42 # kg/m3 # correction to reproduce the elevation difference 
                                                     # in the 2-D thermo-mechanical model
    geotherm_filename                = 'profile_temperature_ridge_centralvalley_GL-Tp1280_speed3-perplex_M1.tz'
    density_profile_filename_ridge   = 'profile_density_ridge_centralvalley_GL-Tp1280_speed3-perplex_M1.rhoz'
    path_used                        = './data/geodyn2d/M1/'
    if printOut:
        print("Density and temperature profiles from 2-D geodynamic model M1 (P-T dependent densities/perplex)")
else:
    deltarho_ridge_correction        = 4.05  # kg/m3 # correction to reproduce the elevation difference 
                                                     # in the 2-D thermo-mechanical model
    geotherm_filename                = 'profile_temperature_ridge_centralvalley_GL-Tp1280_speed3_M2.tz'
    density_profile_filename_ridge   = 'profile_density_ridge_centralvalley_GL-Tp1280_speed3_M2.rhoz'
    path_used                        = './data/geodyn2d/M2/'
    if printOut:
        print("Density and temperature profiles from 2-D geodynamic model M2 (temperature dependent only densities)")

lith1 = geodyn1d.lithosphere('ridge_Oc6_5_NoDiffLayer',printOut=False)
lith1.update_materials(book_reflith)
lith1.get_imposed_geotherm(path_used+geotherm_filename,printOut=printOut)
lith1.get_pressure_density_profile(printOut=False,
                                   filename=path_used+density_profile_filename_ridge,
                                       drho=deltarho_ridge_correction,
                                       zdrho=[6500,125000])

results = {
       'Reference continental crustal density': [],
       'Average continental crustal density': [],
       'Average degree depletion CLM': [],
       'Water depth': [],
       'Sea level': [],
       'dh': []
}

for sea_level_ref in allsea_level_ref:
    for refdensity_crust in allrefdensity_crust:
        for lith_depletion in alllith_depletion: 
            
            # Update the book ------------------------
            book_complith=update_book(book_complith,'rho',refdensity_crust,['UC','MC','LC'])
            book_complith=update_book(book_complith,'deltarho',lith_depletion,['LM1','LM2','LM3'])
            # ----------------------------------------

            print(book_complith)
            
            if printOut:
                print("# Column 2: Reference Continental lithosphere ###############################################################")
            lith2 = geodyn1d.lithosphere(complith,printOut=False)
            lith2.update_materials(book_complith)
            lith2.get_steady_state_geotherm(dz=dz,printOut=printOut)
            lith2.get_pressure_density_profile(printOut=False,path=path+perplex_name)
            pref  = lith2.get_reference_pressure(ref_depth,printOut=False)
            # compute average crustal density for information
            rhoc = lith2.P_rho_DepthProfile.rho[lith2.P_rho_DepthProfile.Z<35000]
            depc = lith2.P_rho_DepthProfile.Z[lith2.P_rho_DepthProfile.Z<35000]
            Averdensity_crust=np.mean(rhoc)
            #-------------------------------------------------------------------
            if use_fixed_waterdepth:
                text = 'WITH FIXED WATER DEPTH  ==========='
                lith1.set_sediment_thickness(0)
                lith1.set_water_depth(water_depth_at_ridge)
                dh,water_depth,sea_level,sediment_thickness = lith1.get_subsidence_with_sediments(pref,
                                                                                                  ref_depth,
                                                                                                  SeaLevelFixed=False,
                                                                                                  printOut=False)
            #-------------------------------------------------------------------
            else:
                text = 'WITH FIXED SEA LEVEL  =========== (semi-analytical solution of the 2-D thermo-mechanical model with water-load)'
                lith1.set_sediment_thickness(0)
                lith1.set_sea_level(sea_level_ref)
                dh,water_depth,sea_level,sediment_thickness = lith1.get_subsidence_with_sediments(pref,
                                                                                                  ref_depth,
                                                                                                  SeaLevelFixed=True,
                                                                                                  printOut=False)
            if printOut:
                print('========= SUBSIDENCE AT THE RIDGE '+text)
                print('Reference crustal density    = {0:6.1f} kg/m3'.format(refdensity_crust))
                print('Average crustal density      = {0:6.1f} kg/m3'.format(Averdensity_crust))
                print('Average degree depletion CLM = {0:6.1f} kg/m3'.format(lith_depletion))
                print('WATER DEPTH                  = {0:6.1f} m'.format(water_depth))
                print('SEA LEVEL                    = {0:6.1f} m'.format(sea_level))
                print('dh                           = {0:6.1f} m'.format(dh))

            results['Reference continental crustal density'].append(refdensity_crust)
            results['Average continental crustal density'].append(Averdensity_crust)
            results['Average degree depletion CLM'].append(lith_depletion)
            results['Water depth'].append(water_depth)
            results['Sea level'].append(sea_level)
            results['dh'].append(dh)




In [ ]:
# Calibration of the crustal density and degree of depletion of the continentlal lithospheric mantle
# and sensitivity on water depth/sea level
#
# Plot of the solutions (Figures 10b and S13)
#

%matplotlib widget

define_rcParams()

write_pdf     = False

sea_level_ref = -400e0    # meters = reference sea level based on average elevation of continents

# Calibration of the crustal density and degree of depletion of the continentlal lithospheric mantle
#
refdensity_mantle = "SP2008_speed3_0.55_bl400_waterload"          # fixed sea level
#refdensity_mantle = "SP2008_speed3_0.55_waterdepth2750_waterload" # fixed water depth the ridge


# Sensitivity on water depth/sea level for temperature dependent only densities
#
# Reference mantle density 3300
#refdensity_mantle = "3300_bl400_waterload"              # fixed sea level
#refdensity_mantle = "3300_waterdepth2750_waterload"     # fixed water depth the ridge
# Reference mantle density 3311
#refdensity_mantle = "3311_bl400_waterload"              # fixed sea level
#refdensity_mantle = "3311_waterdepth2750_waterload"     # fixed water depth the ridge


filename  = "rhoc_lithdepletion_"+str(refdensity_mantle)+".txt"
path_used = './data/isostasy_calibration/'

lines = np.loadtxt(path_used+filename, comments="#", delimiter=" ", unpack=False)
x  = np.asarray(lines[:,0]) # Average depletion buoyancy CLM
y  = np.asarray(lines[:,1]) # Reference crustal density
dh = np.asarray(lines[:,2])

if (refdensity_mantle.find('waterdepth')>0):
    waterdepth = np.asarray(lines[:,3])
    sealevel   = np.asarray(lines[:,4])

    x = x[(waterdepth==2750)]
    y = y[(waterdepth==2750)]
    sealevel = sealevel[(waterdepth==2750)]
    elev     = -1*sealevel

    yy       = np.arange(2750,2960,10) # allrefdensity_crust
    xx       = np.arange(-30,2,2)      # alllith_depletion
    X, Y     = np.meshgrid(xx,yy)
    C        = griddata((x,y),elev,(X,Y),method='linear')
else:
    x    = np.sort(np.unique(x))
    y    = np.sort(np.unique(y))
    X, Y = np.meshgrid(x,y)
    C    = dh.reshape(len(y),len(x))-sea_level_ref

plt.figure(figsize=(20/2.54,20/2.54))
rcParams['axes.labelsize'] = 10


if (refdensity_mantle.find('waterdepth')>0):
    CS = plt.pcolormesh(X,Y,C,cmap='viridis_r')
    cont = plt.contour(X, Y, C, [100,200,300,400,500,600,700,800], colors='k')
else:
    CS = plt.pcolormesh(X,Y,C,cmap='viridis')
    if (refdensity_mantle.find('waterload')>0):
        cont = plt.contour(X, Y, C, [-3150,-3050,-2950,-2850,-2750,-2650,-2550,-2450,-2350], colors='k')
    else:
        cont = plt.contour(X, Y, C, [-2500,-2300,-2100], colors='k')

plt.clabel(cont,fmt='%1.0f')
        
        
plt.xlabel(r'Average depletion buoyancy CLM $kg/m^{3}$')
plt.ylabel(r'Reference crustal density $\rho_{0}$ $kg/m^{3}$')

ax          = plt.gca()
colorbar    = "right"
orientation = {"left": "vertical", "right": "vertical",
               "bottom": "horizontal", "top": "horizontal"}
divider     = make_axes_locatable(ax)
cax         = divider.append_axes(colorbar, size="5%", pad=0.05)
if (refdensity_mantle.find('waterdepth')>0):
    cb      = plt.colorbar(CS,cax=cax,orientation=orientation[colorbar])
    cb.ax.set_ylabel("Elevation of the continents (m)", rotation=270)
    cb.ax.get_yaxis().labelpad = 15
    cb.ax.invert_yaxis() 
else:
    cb      = plt.colorbar(CS,cax=cax,orientation=orientation[colorbar])
    cb.ax.set_ylabel("MOR elevation below sea level (m)", rotation=270)
    cb.ax.get_yaxis().labelpad = 15
    cb.ax.invert_yaxis() 

cb.solids.set_rasterized(True)
if write_pdf:
    whatplot = filename
    plt.savefig("./ps/"+whatplot+".pdf",bbox_inches='tight',dpi=1200)

plt.show()